In [20]:
import sys
import os
import cv2
import math

import numpy as np
print("numpy v%s"%np.__version__)

import pandas as pd
import liblas

import parse_tracklet as pt

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from mpl_toolkits.axes_grid1 import ImageGrid


from image_utils import *
from unwrapper import *

mpl.rcParams['savefig.dpi'] = 320
mpl.rcParams['figure.dpi'] = 320


numpy v1.12.1


In [3]:
data_group_folder = 'data_3'
bag_name = '9'
data_folder_path = '/Users/max/Desktop/carchallenge/car_out/'+data_group_folder+'/'+bag_name
print("Looking in '%s'"%data_folder_path)

Looking in '/Users/max/Desktop/carchallenge/car_out/data_3/9'


In [4]:
csv_filepath = data_folder_path+'/points.csv'
print("Loading csv from '%s'"%csv_filepath)
df = pd.DataFrame.from_csv(csv_filepath)
#df.index = pd.to_datetime(df.index)
print(df.axes[1])

Loading csv from '/Users/max/Desktop/carchallenge/car_out/data_3/9/points.csv'
Index([u'x', u'y', u'z', u'intensity', u'ring'], dtype='object')


In [5]:
print(df.index)

Int64Index([1490994769796104000, 1490994769796104000, 1490994769796104000,
            1490994769796104000, 1490994769796104000, 1490994769796104000,
            1490994769796104000, 1490994769796104000, 1490994769796104000,
            1490994769796104000,
            ...
            1490994781406147000, 1490994781406147000, 1490994781406147000,
            1490994781406147000, 1490994781406147000, 1490994781406147000,
            1490994781406147000, 1490994781406147000, 1490994781406147000,
            1490994781406147000],
           dtype='int64', name=u'timestamp', length=3630085)


In [6]:
xml_filepath = data_folder_path+'/tracklet_labels.xml'
tracklets = pt.parse_xml(xml_filepath)
for i in range(len(tracklets)):
    tracklet = tracklets[i]
    h, w, l = tracklet.size
    print("%s. %s"%((i+1), tracklet.object_type))
    print("    LxWxH:  %s x %s x %s"%(l,w,h))
    print("    Frame0: %s "%(tracklet.first_frame))
    print("    Frames: %s "%(tracklet.num_frames))
    print("    Start:  (%s, %s, %s) "%tuple(tracklet.trans[0]))
    print("    End:  (%s, %s, %s) "%tuple(tracklet.trans[-2]))

Parsing Tracklet file /Users/max/Desktop/carchallenge/car_out/data_3/9/tracklet_labels.xml
File contains 1 Tracklets
Loaded 1 Tracklets
1. Car
    LxWxH:  4.5212 x 1.7018 x 1.397
    Frame0: 0 
    Frames: 279 
    Start:  (40.83528, 5.155688, 0.3379) 
    End:  (-19.128551, 3.144657, -0.9701) 


In [7]:
def get_centroid(trans_xyz, obj_lwh):
    return np.array(trans_xyz)# + np.array(obj_lwh)/2.0

tracklet_centroids = []
for tracklet in tracklets:
    for i in range(tracklet.num_frames):
        frame_index = i + tracklet.first_frame
        trans = tracklet.trans[i]
        centroid = get_centroid(trans_xyz=trans, obj_lwh=tracklet.size) 
        tracklet_centroids.append(centroid)

print("Loaded %s centroids"%len(tracklet_centroids))

Loaded 279 centroids


In [8]:
# Group the data per time period
df_seconds = df.groupby(df.index)#.map(lambda t: t/50))#t.microsecond/500))
print(len(df_seconds), df_seconds)
print(df_seconds.size())

(117, <pandas.core.groupby.DataFrameGroupBy object at 0x103f62a10>)
timestamp
1490994769796104000    32514
1490994769896199000    32482
1490994769996290000    32497
1490994770096338000    32525
1490994770196425000    32581
1490994770296558000    32472
1490994770396599000    32550
1490994770496697000    32564
1490994770596821000    32569
1490994770696885000    32679
1490994770796986000    32789
1490994770897057000    32879
1490994770997142000    32821
1490994771097228000    32833
1490994771197321000    32816
1490994771297419000    32829
1490994771397478000    32729
1490994771497588000    32710
1490994771597685000    32723
1490994771697773000    32687
1490994771797830000    32590
1490994771897952000    32533
1490994771998016000    32577
1490994772098111000    32507
1490994772198173000    32468
1490994772298274000    32168
1490994772398358000    32352
1490994772498434000    32564
1490994772598520000    32737
1490994772698623000    32278
                       ...  
1490994778503636000    

In [9]:
"""
res:        (float) Desired resolution in metres. Each output pixel will represent a region res x res in size.
side_range: (tuple of two floats) (-left, right) in metres left and right limits of rectangle to look at.
fwd_range:  (tuple of two floats) (-behind, front) in metres back and front limits of rectangle to look at.
height_range: (tuple of two floats) (min, max) heights (in metres) relative to the origin.
            All height values will be clipped to this min and max value, such that anything 
            below min will be truncated to min, and the same for values above max.
"""

def points_to_image(points, 
                    res=0.1,
                    side_range=(-50.0, 50.0),
                    fwd_range=(-50.0, 50.0),
                    height_range=(-20.0, 20.0)):
    x_points = points['x']
    y_points = points['y']
    z_points = points['z']
    
    #f_filt = np.logical_and((x_points > fwd_range[0]), (x_points < fwd_range[1]))
    #s_filt = np.logical_and((y_points > -side_range[1]), (y_points < -side_range[0]))
    ##h_filt = np.logical_and((z_points > height_range[1]), (z_points < height_range[0]))
    #print(min(z_points), max(z_points))
    #filter = np.logical_and(f_filt, s_filt)
    #indices = np.argwhere(filter).flatten()
    #print("Keeping %s/%s points (%s%%)" % (len(indices), len(x_points), round(len(indices)*100.0/len(x_points))))
    #
    #x_points = x_points[indices]
    #y_points = y_points[indices]
    #z_points = z_points[indices]
    
    # CONVERT TO PIXEL POSITION VALUES - Based on resolution
    x_img = (x_points / res).astype(np.int32)  # x axis is -y in LIDAR
    y_img = (y_points / res).astype(np.int32)  # y axis is -x in LIDAR

    # SHIFT PIXELS TO HAVE MINIMUM BE (0,0)
    # floor & ceil used to prevent anything being rounded to below 0 after shift
    #x_img -= int(np.floor(side_range[0] / res))
    #y_img += int(np.ceil(fwd_range[1] / res))
    
    return [x_img, y_img]

     # CLIP HEIGHT VALUES - to between min and max heights
    pixel_values = np.clip(a=z_points,
                           a_min=height_range[0],
                           a_max=height_range[1])

    # RESCALE THE HEIGHT VALUES - to be between the range 0-255
    pixel_values = scale_to_255(pixel_values,
                                min=height_range[0],
                                max=height_range[1])

    # INITIALIZE EMPTY ARRAY - of the dimensions we want
    x_max = 1 + int((side_range[1] - side_range[0]) / res)
    y_max = 1 + int((fwd_range[1] - fwd_range[0]) / res)
    im = np.zeros([y_max, x_max], dtype=np.uint8)

    # FILL PIXEL VALUES IN IMAGE ARRAY
    im[y_img, x_img] = pixel_values

    return im
    
#im = points_to_image(df)
#break
#for second, group in df_seconds:
#    im = points_to_image(group)
#    plt.imshow(im)
    #images.append(im)
    #images.append(im.T)
    #if len(images) > 3:
    #    break
#image_grid(images)

In [21]:
"""
CONSTANTS:
    V_RES_DEG: (float (deg)) Vertical angular res. Affects output image height.
    H_RES_DEG: (float (deg)) Horizontal angular res. Affects output image width.
    V_FOV_DEG: (floats) Vertical field of view in degrees. (-min_angle, max_angle)
    Y_FUDGE: (float) A hacky fudge factor to use if the theoretical image height doesnt match the actual data.

"""
# degree to radians
D2R = (np.pi / 180)

V_RES_DEG = 1.0 #0.9
H_RES_DEG = 0.9#75 #0.3
V_FOV_DEG = (-30.67, 10.67)
V_FOV_TOTAL_DEG = -V_FOV_DEG[0] + V_FOV_DEG[1]
Y_FUDGE = 0.0

# CONVERT TO RADIANS
V_RES_RAD = V_RES_DEG * D2R
H_RES_RAD = H_RES_DEG * D2R
V_FOV_RAD = (V_FOV_DEG[0] * D2R, V_FOV_DEG[1] * D2R)
V_FOV_TOTAL_RAD = V_FOV_TOTAL_DEG * D2R

D_PLANE = (V_FOV_TOTAL_DEG / V_RES_DEG) / V_FOV_TOTAL_RAD
H_AVOVE_AND_BELOW = D_PLANE * (np.tan(-V_FOV_RAD[0]) + np.tan(V_FOV_RAD[1]))

# THEORETICAL LIMITS FOR IMAGE
Y_MIN = -((V_FOV_DEG[1] / V_RES_DEG) + Y_FUDGE)
Y_MAX = int(np.ceil(H_AVOVE_AND_BELOW + Y_FUDGE))
X_MIN = -180.0 / H_RES_DEG
X_MAX = int(np.ceil(360.0 / H_RES_DEG))
IMG_DIMENSIONS = [Y_MAX + 1, X_MAX + 1]

In [22]:
# Image helpers

def image_grid(images, n_cols=4):
    n_rows = int(math.ceil(len(images)/float(n_cols)))
    fig = plt.figure(1, (4., 4.))
    grid = ImageGrid(fig, 111, nrows_ncols=(n_rows, n_cols), axes_pad=0.1)
    
    for i in range(n_rows*n_cols):
        if i < len(images):
            grid[i].imshow(images[i])
        grid[i].axis[:].toggle(all=False)

In [23]:
TIMESTEPS = df.index.unique().tolist()


In [24]:
def draw_centroid(img, timestamp, scale=1.0):
    img = np.copy(img)
    centroid = get_first_centroid_for(timestamp)
    x = centroid[0]
    y = centroid[1]
    z = centroid[2]
    d = distance(x, y, z)
    x, y = unwrap_point(x, y, z, d)
    r = (200/d)#*10
    cv2.circle(img, (int(x*scale), int(y*scale*1.2)), int(r), (0,255,0), 1)
    return img

def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping pixel values [0, 255] to their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array( [((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)] ).astype("uint8")
    
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [25]:
# Interpolate centroids

def get_num_centroids():
    return len(tracklet_centroids)

def get_first_centroid_for(timestamp):
    i = TIMESTEPS.index(timestamp)
    #(timestamp.value/get_data_end())*get_num_centroids()
    #centroid_index = int(np.floor(np.float(timestamp.value - get_data_start())/get_centroid_duration()))
    percent = (float(i)/len(TIMESTEPS))
    centroid_index = int( (percent)*get_num_centroids() )
    index_max = get_num_centroids() - 1
    if centroid_index >= index_max:
        centroid_index = index_max
    #print("centroid_index: %s"%centroid_index)
    return tracklet_centroids[centroid_index]

In [82]:
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis/np.sqrt(np.dot(axis, axis))
    a = np.cos(theta/2.0)
    b, c, d = -axis*np.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

def rotate_around_center(x, y, z, theta):
    rm = rotation_matrix([0.0, 0.0, 1.0], theta)
    return np.nan_to_num(np.dot(np.column_stack((x,y,z)), rm.T))
    
def unwrap_point(x, y, z, d, angle_offset=0.0):
    theta = angle_offset * float(D2R)
    if theta != 0:
        x, y, z = rotate_around_center(x,y,z, theta).T
    # MAP TO CYLINDER
    # angle_offset
    
    x_img = np.arctan2(y, x) / H_RES_RAD
    y_img = -(np.arctan2(z, d) / V_RES_RAD)

    # SHIFT COORDINATES TO MAKE 0,0 THE MINIMUM
    x_img = np.trunc(-x_img - X_MIN).astype(np.int32)
    y_img = np.trunc(y_img - Y_MIN).astype(np.int32)

    return x_img, y_img



def point_cloud_to_panorama(points,
                            d_range=(-10.0, 200.0),
                            angle_offset=0.0,
                            ):
    # map distance relative to origin
    # z is up, x is forward, y is left
    points['map_distance'] = distance(points['x'], points['y'])  # np.sqrt(points['x']**2 + points['y']**2)
    # print("map_distance", points['map_distance'].shape)
    # print("map_distance min: %s    max:  %s"%(points['map_distance'].min(), points['map_distance'].max()))

    # Filter by distance
    points = points[points['map_distance'] > d_range[0]]
    points = points[points['map_distance'] < d_range[1]]
    # print("filtered points", points.shape)

    d_points = points['map_distance']
    r_points = np.sqrt(points['intensity'])

    x_points, y_points = unwrap_point(points['x'], points['y'], points['z'], d_points, angle_offset=angle_offset)

    xy_points = [x_points, y_points]

    img_distance = array_to_image(d_points, xy_points, IMG_DIMENSIONS)
    img_intensity = array_to_image(r_points, xy_points, IMG_DIMENSIONS)

    return img_distance, img_intensity


In [83]:
IMAGE_LAG_NUM=1
PAST_IMAGE_WEIGHT=0.4
IMG_ANGLE = 45.0

past_images = {"intensity": LaggingImage(past_image_limit=IMAGE_LAG_NUM, past_image_weight=PAST_IMAGE_WEIGHT),
               "distance": LaggingImage(past_image_limit=IMAGE_LAG_NUM, past_image_weight=PAST_IMAGE_WEIGHT)}
                #, "depth_map": []}
frames = {"distance": [], "intensity": []}

for second, group in df_seconds:
    img_dist, img_intensity = point_cloud_to_panorama(group, d_range=(0.0, 40.0), angle_offset=IMG_ANGLE)

    past_images["distance"].add(img_dist)
    past_images["intensity"].add(img_intensity)
    #past_images["depth_map"].add(img_depth)
    #print(img_dist.shape, np.mean([img_dist, img_dist], axis=0).shape)

    ts_start = group.axes[0][0]
    for display in ["distance", "intensity"]:
        img = past_images[display].get_lagged_image(scale_up=1.5)
        #print(img.__class__, img)
        #img = cv2.applyColorMap(img, cv2.COLORMAP_JET)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        if display == "intensity":
            img = adjust_gamma(img, 1.2)
        #img = cv2.bitwise_not(img)
        img = draw_centroid(img, ts_start, scale=1.5)
        frames[display].append(img)
        #plt.imshow()
        #show(img)#, file_name=("unwrapped_pointcloud_%s_%s.png"%(display, second)))
        #cv2.imwrite("unwrapped_pointcloud_%s_%s.png"%(display, second), img)
        #break
    #break

In [73]:
#for i in range(10):
#    img = point_cloud_to_panorama(group,
#                                  im_format='float',
#                                  d_range=(10*i, 10+10*i),
#                                  v_res=1.0,
#                                  h_res=0.75)
#    
#    show(img)
#
#

In [74]:
print(len(frames["distance"]))

117


In [77]:
from moviepy.editor import ImageSequenceClip, clips_array

FPS=13

intensity_animation = ImageSequenceClip(frames['intensity'], fps=FPS)
distance_animation = ImageSequenceClip(frames['distance'], fps=FPS)
h, w, d = frames['distance'][0].shape
blank = np.zeros([h/2, w, d], dtype=np.uint8)
blank[:,:] = [255,255,255]
blank = ImageSequenceClip([blank]*(len(frames['intensity'])), fps=FPS)

final_clip = clips_array([[intensity_animation], [blank], [distance_animation]])
filename = "unwrapped_pointcloud_%s_%s_isqrt_gamma_w%s.webm"%(data_group_folder, bag_name, PAST_IMAGE_WEIGHT)
final_clip.write_videofile(filename, codec='libvpx')
final_clip.ipython_display(loop=True, autoplay=True)

[MoviePy] >>>> Building video unwrapped_pointcloud_data_3_9_isqrt_gamma_w0.4.webm
[MoviePy] Writing video unwrapped_pointcloud_data_3_9_isqrt_gamma_w0.4.webm


100%|██████████| 117/117 [00:02<00:00, 54.40it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: unwrapped_pointcloud_data_3_9_isqrt_gamma_w0.4.webm 



100%|██████████| 117/117 [00:00<00:00, 488.74it/s]
